In [ ]:
# List tag entitas
entities = [
    "DAT", "EVT", "GPE", "LOC", "PER", "PRD", 
    "PRC", "CRD", "TIM", "NOR", 
    "ECONOMIC_INDICATOR", "ASSET_CLASS"
]

# Membuat filter encoding
label_to_id = {"O": 0}  # Mulai dengan tag 'O'
id_to_label = {0: "O"}  # Untuk decoding nanti
current_id = 1

for entity in entities:
    label_to_id[f"B-{entity}"] = current_id
    id_to_label[current_id] = f"B-{entity}"
    current_id += 1
    
    label_to_id[f"I-{entity}"] = current_id
    id_to_label[current_id] = f"I-{entity}"
    current_id += 1

# Hasilnya
print("Label to ID Mapping:", label_to_id)
print("ID to Label Mapping:", id_to_label)
